In [1]:
import os
import glob
import random
from pathlib import Path
from datetime import datetime

import cv2
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F

import albumentations as A
from albumentations.pytorch import ToTensorV2


LR   = 5e-5
SEED = 5
EPOCHS = 10
BATCH_SIZE  = 32
NUM_WORKERS =  4

MODEL_NAME = f'nfnet_l2-{SEED:04d}'

random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = False

cv2.setNumThreads(4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('DEVICE =', device)

DEVICE = cuda


In [2]:
df = pd.read_csv('train.csv')
df['class'] = df['class'].values.astype(int)
df.head()

,ID_img,class
0,2138.jpg,4
1,2139.jpg,6
2,2140.jpg,3
3,2141.jpg,6
4,2142.jpg,3


In [3]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.0)

img_model = timm.create_model('eca_nfnet_l2', pretrained=True)
in_features = img_model.head.fc.in_features
img_model.head.fc = nn.Linear(in_features, 8)

img_model.head.fc.apply(init_weights)

Linear(in_features=3072, out_features=8, bias=True)

In [4]:
class Model(nn.Module):

    def __init__(self, 
                 img_model,
                 classes: int = df['class'].nunique(),
                 in_features:int = in_features):
        
        super().__init__()
        self.in_features = in_features
        self.img_model = img_model

    def forward(self, x):
        return self.img_model(x)


model = Model(img_model)

In [5]:
transform_train = A.Compose([
    A.LongestMaxSize(160, interpolation=cv2.INTER_AREA),
    A.PadIfNeeded(160, 160),
    A.CoarseDropout(p=0.2),
    A.VerticalFlip(p=0.1),
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.1),
    A.ColorJitter(p=0.5),
    A.ImageCompression(quality_lower=30, quality_upper=100, p=0.5),
    A.OneOf([
        A.GaussNoise(var_limit=5, p=0.1),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05,
                       scale_limit=0.10,
                       rotate_limit=7.0,
                       p=0.3),
    A.OneOf([
        A.OpticalDistortion(p=1),
        A.Perspective(p=1),
        A.GridDistortion(p=1),
    ], p=0.2),
    A.OneOf([
        A.CLAHE(clip_limit=2),
        A.Sharpen(p=1),
        A.Emboss(p=1),
        A.RandomBrightnessContrast(p=1),
    ], p=0.2),
    A.Normalize(mean=img_model.pretrained_cfg['mean'],
                std =img_model.pretrained_cfg['std']),
    ToTensorV2()
])

transform_val = A.Compose([
    A.LongestMaxSize(160, interpolation=cv2.INTER_AREA),
    A.PadIfNeeded(160, 160),
    A.Normalize(mean=img_model.pretrained_cfg['mean'],
                std =img_model.pretrained_cfg['std']),
    ToTensorV2()
])

In [6]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe: pd.DataFrame,
                 transforms: A.Compose = None,
                 mode: str = 'train'):

        self.mode = mode
        self.cls  = dataframe['class'].values
        self.imgs = dataframe['ID_img'].apply(lambda x: os.path.join(mode, f'{x}')).values
        self.transforms = transforms

    def __getitem__(self, idx) -> dict:

        image_path = self.imgs[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        item = {'image': image}
        if self.transforms is not None:
            item = self.transforms(**item)
        
        item['trg_cls']   = torch.tensor(self.cls[idx]).long()

        return item

    def __len__(self) -> int:
        return len(self.imgs)


dataset_train = Dataset(df, transforms=transform_train, mode='train')
dataset_valid = Dataset(df, transforms=transform_val, mode='train')

loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE,
                                           num_workers=NUM_WORKERS, shuffle=True,
                                           drop_last=True, pin_memory=True)

loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=BATCH_SIZE,
                                           num_workers=NUM_WORKERS, shuffle=False,
                                           drop_last=False, pin_memory=True)

In [7]:
model.cuda()

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, epochs=EPOCHS, max_lr=LR,
                                                   div_factor=25.0, final_div_factor=10.0, steps_per_epoch=1)

best_cnt = 0
best_f1  = 0.0

weight = len(df) / (len(np.unique(df['class'].values)) * np.bincount(df['class'].values))
weight = torch.FloatTensor(weight)
weight = torch.nan_to_num(weight, posinf=1.0, neginf=1.0)
weight = weight.cuda()

for epoch in range(EPOCHS):

    criterion  = nn.CrossEntropyLoss(weight)
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Start epoch {epoch + 1} at {datetime.now().strftime('%H:%M:%S')}, lr={current_lr:0.8f}")

    loss_cls = []
    model.train()
    optimizer.zero_grad()
    for batch in loader_train:
        batch = {k:v.cuda() for k, v in batch.items()}
        
        pred = model(batch['image'])
        loss = criterion(pred, batch['trg_cls'])

        loss_cls.append(loss.item())

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()

    lr_scheduler.step()

    torch.cuda.empty_cache()
    model.eval()

    val_cls = []
    trg_cls = []
    with torch.no_grad():
        for batch in loader_valid:
            batch = {k:v if k.startswith('trg_') else v.cuda() for k, v in batch.items()}
            pred = model(batch['image'])

            trg_cls.extend(batch['trg_cls'].numpy().tolist())
            val_cls.extend(pred.argmax(dim=1).cpu().numpy().tolist())

    f1_cls  = f1_score(trg_cls, val_cls, zero_division=0, average='macro')

    print(f"  train loss: cls = {min(9.9999, np.mean(loss_cls)):6.4f}")
    print(f"  valid f1:   cls = {f1_cls:6.4f}")

    best_cnt += 1
    if f1_cls >= best_f1:
        best_cnt = 0
        best_f1 = f1_cls
        torch.save(model.state_dict(), f"{MODEL_NAME}.pth")
        torch.save(model.state_dict(), f"{MODEL_NAME}_{epoch}.pth")
        print("Saved best model!")
    elif best_cnt > EPOCHS // 5:
        best_cnt = 0
        print("Loading best model weights!")
        model.load_state_dict(torch.load(f"{MODEL_NAME}.pth", map_location=device))        

Start epoch 1 at 21:42:01, lr=0.00000200
  train loss: cls = 1.9619
  valid f1:   cls = 0.5292
Saved best model!
Start epoch 2 at 21:44:22, lr=0.00002600
  train loss: cls = 0.5773
  valid f1:   cls = 0.9714
Saved best model!
Start epoch 3 at 21:46:50, lr=0.00005000
  train loss: cls = 0.2016
  valid f1:   cls = 0.9863
Saved best model!
Start epoch 4 at 21:49:28, lr=0.00004753
  train loss: cls = 0.1072
  valid f1:   cls = 0.9973
Saved best model!
Start epoch 5 at 21:52:05, lr=0.00004062
  train loss: cls = 0.0721
  valid f1:   cls = 0.9973
Start epoch 6 at 21:54:40, lr=0.00003064
  train loss: cls = 0.0473
  valid f1:   cls = 0.9990
Saved best model!
Start epoch 7 at 21:56:57, lr=0.00001956
  train loss: cls = 0.0298
  valid f1:   cls = 0.9997
Saved best model!
Start epoch 8 at 21:59:00, lr=0.00000958
  train loss: cls = 0.0320
  valid f1:   cls = 0.9992
Start epoch 9 at 22:01:19, lr=0.00000267
  train loss: cls = 0.0314
  valid f1:   cls = 0.9997
Saved best model!
Start epoch 10 at 2